Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed PyTorch with DistributedDataParallel
In this tutorial, you will train a PyTorch model on the [CIFAR10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset using distributed training with PyTorch's `DistributedDataParallel` module across a Azure Stack Hub CPU Kubernetes cluster. The training dataset are stored on Azure Machine Learning datastore backed up by Azure Stack Hub Storage account.

## Prerequisite

*     A Kubernetes cluster deployed on Azure Stack Hub, connected to Azure through ARC.
     
   For details on how to deploy kubernetes cluster on Azure Stack Hub and enabling ARC connection to Azure, please follow [this guide](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md)
  

*     Datastore setup in Azure Machine Learning workspace backed up by Azure Stack Hub storage account.

   [This document](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md) is a detailed guide on how to create Azure Machine Learning workspace, create a  Azure Stack Hub Storage account, and setup datastore in AML workspace backed by ASH storage account.


*      Last but not least, you need to be able to run a Notebook. 

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at [here](https://github.com/Azure/MachineLearningNotebooks) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [2]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: sl-ash2-mal
Azure region: eastus
Subscription id: 6b736da6-3246-44dd-a0b8-b5e95484633d
Resource group: sl-ash2


## Prepare dataset

You may download cifar10 dataset from [cifar10-data](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz). Create folder "cifar10-data" under working directory of this notebook, then  copy "cifar-10-python.tar.gz" to folder "cifar10-data". The following cell will upload "cifar-10-python.tar.gz" to datastore of the workspace, and finally registered as dataset in the workspace. 

Upload and dataset registration take about 3 mins.

To set up datastore using an azure stack hub storage account, please refer to [Train_azure_arc](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md). To register the dataset manually, please refer to this [video](https://msit.microsoftstream.com/video/51f7a3ff-0400-b9eb-2703-f1eb38bc6232)


In [3]:
import os
from azureml.core import Datastore, Dataset

dataset_name = 'cifar10'
datastore_name = "ashstore"

if dataset_name not  in ws.datasets:
    datastore =  Datastore.get(ws, datastore_name)
    
    src_dir, target_path = 'cifar10-data', 'cifar10-data-ash' #assuming cifar-10-python.tar.gz is in folder cifar10-data
    
    # upload data from local to AML datastore:
    datastore.upload(src_dir, target_path)

    # register data uploaded as AML dataset:
    datastore_paths = [(datastore, target_path)]
    cifar_ds = Dataset.File.from_files(path=datastore_paths)
    cifar_ds.register(ws, dataset_name, "CIFAR-10 images from https://www.cs.toronto.edu/~kriz/cifar.html")
        
dataset_ash = ws.datasets[dataset_name]

## Create or attach existing ArcKubernetesCompute

The attaching code here depends  python package azureml-contrib-k8s which current is in private preview. Install private preview branch of AzureML SDK by running following command (private preview):

<pre>
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-contrib-k8s-preview/D58E86006C65 azureml-contrib-k8s
</pre>

Attaching ASH cluster the first time may take 7 minutes. It will be much faster after first attachment.

In [6]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute
from azureml.core import ComputeTarget

resource_id = "/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourceGroups/sl-ash2/providers/Microsoft.Kubernetes/connectedClusters/sl-d2-o-arc"

attach_config = ArcKubernetesCompute.attach_configuration(
    resource_id= resource_id,
)

try:
    attach_name = "sl-d2-o-arc"
    arcK_target_result = ArcKubernetesCompute.attach(ws, attach_name, attach_config)
    arcK_target_result.wait_for_completion(show_output=True)
    print('arc attach  success')
except ComputeTargetException as e:
    print(e)
    print('arc attach  failed')

attach_name = "nc6"
arcK_target = ComputeTarget(ws, attach_name)

SucceededProvisioning operation finished, operation "Succeeded"
arc attach  success


## Configure the training job and Submit a run


### Create an experiement

In [7]:
from azureml.core import Experiment

experiment_name = 'pytorch-cifar-distr'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

In [8]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-cpu', file_path = 'pytorch-script/conda_dependencies.yml')

# Specify a CPU base image
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04'


### Configure the training job: torch.distributed with GLOO backend

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In order to run a distributed PyTorch job with **torch.distributed** using the GLOO backend, create a `PyTorchConfiguration` and pass it to the `distributed_job_config` parameter of the ScriptRunConfig constructor. Specify `communication_backend='Gloo'` in the PyTorchConfiguration. The below code will configure node_count = 2. These is the number of worker nodes. The number of  distributed jobs will be 3 if one master node is used.  GLOO backend which is recommended backend for communications between CPUs.

Tthe script for distributed training of CIFAR10 is already provided for you at `pytorch-script/cifar_dist_main.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

With node_count=2, training for one epoch may take 20 mins with vm size comparable to Standard_DS3_v2

In [9]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core import Dataset
import os

dataset_ash = Dataset.get_by_name(ws, name=dataset_name)
args = [
        '--data-folder', dataset_ash.as_mount(),
        '--dist-backend', 'gloo',
       '--epochs', 1 #20
           ]

distributed_job_config=PyTorchConfiguration(communication_backend='Gloo', node_count=2) #configuring AML pytorch config

project_folder = "pytorch-script"
run_script = "cifar_dist_main.py"
src = ScriptRunConfig(
                     source_directory=project_folder,
                      script=run_script,
                      arguments=args,
                      compute_target=arcK_target,
                      environment=pytorch_env,
                      distributed_job_config=distributed_job_config)

### Submit job
Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [10]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True) # this provides a verbose log

RunId: pytorch-cifar-distr_1613693036_9d4025fd
Web View: https://ml.azure.com/experiments/pytorch-cifar-distr/runs/pytorch-cifar-distr_1613693036_9d4025fd?wsid=/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourcegroups/sl-ash2/workspaces/sl-ash2-mal

Streaming azureml-logs/55_azureml-execution-tvmps_3961494842bf7a51f8ef586a1b7dd63c312f2ca5ade787f9dcfe9df0f7d79a60_d.txt

2021-02-19T00:08:06Z Starting output-watcher...
2021-02-19T00:08:06Z IsDedicatedCompute == True, won't poll for Low Pri Preemption

Streaming azureml-logs/65_job_prep-tvmps_3961494842bf7a51f8ef586a1b7dd63c312f2ca5ade787f9dcfe9df0f7d79a60_d.txt

[2021-02-19T00:08:31.345165] Entering job preparation.
[2021-02-19T00:08:31.893023] Starting job preparation.
[2021-02-19T00:08:31.893062] Extracting the control code.
[2021-02-19T00:08:31.936700] fetching and extracting the control code on master node.
[2021-02-19T00:08:31.936728] Starting extract_project.
[2021-02-19T00:08:31.936760] Starting to extract zip file.
[2021-

96 196 Loss: 1.867 | Acc: 30.034% (3729/12416)
97 196 Loss: 1.864 | Acc: 30.118% (3778/12544)
98 196 Loss: 1.861 | Acc: 30.169% (3823/12672)
99 196 Loss: 1.859 | Acc: 30.227% (3869/12800)
100 196 Loss: 1.857 | Acc: 30.306% (3918/12928)
101 196 Loss: 1.855 | Acc: 30.323% (3959/13056)
102 196 Loss: 1.853 | Acc: 30.438% (4013/13184)
103 196 Loss: 1.852 | Acc: 30.476% (4057/13312)
104 196 Loss: 1.849 | Acc: 30.618% (4115/13440)
105 196 Loss: 1.847 | Acc: 30.727% (4169/13568)
106 196 Loss: 1.845 | Acc: 30.834% (4223/13696)
107 196 Loss: 1.842 | Acc: 30.961% (4280/13824)
108 196 Loss: 1.840 | Acc: 31.064% (4334/13952)
109 196 Loss: 1.838 | Acc: 31.179% (4390/14080)
110 196 Loss: 1.836 | Acc: 31.194% (4432/14208)
111 196 Loss: 1.834 | Acc: 31.250% (4480/14336)
112 196 Loss: 1.832 | Acc: 31.305% (4528/14464)
113 196 Loss: 1.830 | Acc: 31.380% (4579/14592)
114 196 Loss: 1.829 | Acc: 31.393% (4621/14720)
115 196 Loss: 1.828 | Acc: 31.472% (4673/14848)
116 196 Loss: 1.828 | Acc: 31.544% (4724/149

{'runId': 'pytorch-cifar-distr_1613693036_9d4025fd',
 'target': 'nc6',
 'status': 'Completed',
 'startTimeUtc': '2021-02-19T00:08:05.413047Z',
 'endTimeUtc': '2021-02-19T00:20:42.240971Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'a89a5286-de7e-4bc6-9986-8bb8ab74f679',
  'azureml.git.repository_uri': 'git@github.com:lisongshan007/AML-Kubernetes.git',
  'mlflow.source.git.repoURL': 'git@github.com:lisongshan007/AML-Kubernetes.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '9af78d232734dace4df94ed28a5e6888ed3905e4',
  'mlflow.source.git.commit': '9af78d232734dace4df94ed28a5e6888ed3905e4',
  'azureml.git.dirty': 'False',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '24d86baf-41f9-40e1-ae0d-5ae8b4c9cdfc'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__f1c41f8b', 'mecha

### Register the model

In [11]:

register_model_name = 'cifar10torch'
model = run.register_model(model_name=register_model_name, model_path='outputs/cifar10torch.pt')

The machine learning model named "cifar10torch" should be registered in your AML workspace.

## Test Registered Model

To test the trained model, you can create (or use existing) a AKS cluster for serving the model using AML deployment

In [12]:
from azureml.core import Environment, Workspace, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute_target import ComputeTargetException
import numpy as np
import json

### Provision the AKS Cluster

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it. It may take 5 mins to create a new AKS cluster.

In [15]:
ws = Workspace.from_config()

# Choose a name for your AKS cluster
aks_name = 'aks-service-2'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    is_new_compute  = False
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)
    is_new_compute  = True
    
print("using compute target: ", aks_target.name)

Found existing cluster, use it.
using compute target:  aks-service-2


### Deploy the model

In [21]:
env = Environment.from_conda_specification(name='pytorch-cifar', file_path='pytorch-script/conda_dependencies.yml')

inference_config = InferenceConfig(entry_script='score_pytorch.py', environment=env)
deploy_config = AksWebservice.deploy_configuration()

#register_model_name = "cifar10torch80"
model = ws.models[register_model_name]
service_name = 'cifartorchservice4'

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deploy_config,
                       deployment_target=aks_target,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......
Succeeded
AKS service creation operation finished, operation "Succeeded"


### Test with Inputs:

For testing purpose, first five images from test batch are extracted. Let's take a look these pictures: 

In [22]:
from PIL import Image

img_files = ["test_img_0_cat.jpg", "test_img_1_ship.jpg","test_img_2_ship.jpg","test_img_3_plane.jpg","test_img_4_frog.jpg"]

for img_file in img_files:
    image = Image.open("test_imgs/{}".format(img_file))
    image.show()

After some data process, these five images are converted to json as input for the trained model. The outputs are logits for each class per image:

In [23]:
with open("cifar_test_input_pytorch.json", "r") as fp:
    inputs_json = json.load(fp)
    
inputs = json.dumps(inputs_json)
resp = service.run(inputs)
predicts = resp["predicts"]
print(predicts)

[[-0.5153380632400513, -1.8001810312271118, 0.9319053292274475, 1.1880533695220947, 0.8720941543579102, 0.5881021022796631, 1.0242440700531006, 0.7531565427780151, -1.9846534729003906, -1.846501350402832], [5.626550674438477, 5.266762733459473, -0.9428859949111938, -3.697542667388916, -2.906374454498291, -6.4479451179504395, -2.991352081298828, -2.7618331909179688, 5.915030479431152, 2.597166061401367], [4.207612991333008, 1.9836726188659668, 0.22895176708698273, -2.0511369705200195, -1.7314388751983643, -4.231654167175293, -2.3347039222717285, -1.3680027723312378, 3.997978687286377, 1.2459831237792969], [3.6119861602783203, 2.680774450302124, 0.2790420949459076, -1.8072761297225952, -2.1849260330200195, -3.877857208251953, -2.5081701278686523, -1.5793991088867188, 3.8795742988586426, 1.4604357481002808], [-2.149258613586426, -3.239884614944458, 2.0069427490234375, 1.7941792011260986, 2.4163215160369873, 0.6278948783874512, 3.9332661628723145, 0.04551582783460617, -3.313084125518799, -

Then you can easily get the predictions of labels:

In [24]:
import numpy as np
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
np_predicts = np.array(predicts)
pred_indexes = np.argmax(np_predicts, 1)

predict_labels = [classes[i] for i in pred_indexes]
print(predict_labels)

['cat', 'ship', 'plane', 'ship', 'frog']


### Delete the newly created cluster

Note: This is important if you wish to avoid the cost of this cluster

In [25]:
if is_new_compute:
    aks_target.delete()
    service.delete()

## Next Steps

1. Learn how to [download model then upload to Azure Storage blobs](../AML-model-download-upload.ipynb)
2. Learn how to [inference using KFServing with model in Azure Storage Blobs](https://aka.ms/kfas)
3. Learn Pipeline Steps with [Object Segmentation](../object-segmentation-on-azure-stack/)